# Imports

In [0]:
from numpy.random import seed
seed(888)
from tensorflow import set_random_seed
set_random_seed(404)

In [0]:
import os
import numpy as np
import tensorflow as tf
import itertools

import keras
from keras.datasets import cifar10 # cifar10 data web : https://www.cs.toronto.edu/~kriz/cifar.html
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout

from IPython.display import display
from keras.preprocessing.image import array_to_img
from keras.callbacks import TensorBoard

from time import strftime

from sklearn.metrics import confusion_matrix

import matplotlib.pyplot as plt

%matplotlib inline

# Constants

In [0]:
LOG_DIR = 'tensorboard_cifar_logs/'

LABEL_NAMES = ['Plane', 'Car', 'Bird', 'Cat', 'Deer', 'Dog', 'Frog', 'Horse', 'Ship', 'Truck']
IMAGE_WIDTH = 32
IMAGE_HEIGHT = 32
IMAGE_PIXEL = IMAGE_WIDTH * IMAGE_HEIGHT
COLOR_CHANNELS = 3
TOTAL_INPUTS = IMAGE_PIXEL * COLOR_CHANNELS
NR_CLASSES = 10

VALIDATION_SIZE = 10000
SMALL_TRAIN_SIZE = 1000

# Get Data

In [0]:
(x_train_all, y_train_all), (x_test, y_test) = cifar10.load_data()

In [0]:
type(cifar10)
type(x_train_all)

# Explore the Data

In [0]:
x_train_all[0] # gives image in array

In [0]:
pic = array_to_img(x_train_all[7]) # making array img into img
display(pic)

In [0]:
y_train_all.shape # total 50,000 entries

In [0]:
y_train_all[7][0] # in cs.torronto web 7th is horse

In [0]:
LABEL_NAMES[y_train_all[7][0]]

In [0]:
plt.imshow(x_train_all[4])
plt.xlabel(LABEL_NAMES[y_train_all[4][0]], fontsize=15)
plt.show()

In [0]:
plt.figure(figsize=(15, 5))

for i in range(10):
  plt.subplot(1, 10, i+1)
  plt.yticks([])
  plt.xticks([])
  plt.xlabel(LABEL_NAMES[y_train_all[i][0]], fontsize=14)
  plt.imshow(x_train_all[i])

In [0]:
# x_train_all[0] # format of the first image
x_train_all[0].shape

In [0]:
nr_images, x, y, c = x_train_all.shape
print(f'images = {nr_images} \t| width = {x}, \t| height = {y} \t| channels ={c}') # channels = color channels per pixel

In [0]:
x_test.shape

# Preprocess Data

In [0]:
x_train_all[0][0][0] # RGB values(diving deeper in image into array of array)

In [0]:
x_train_all[0][0][0][0]

In [0]:
type(x_train_all[0][0][0][0])  # 8 bit unsigned integer (uint)

In [0]:
# all values in array will be in 0 and 1 also there will some conversion from int to float as it is decimals | bringing range down to adjust learning rate
x_train_all, x_test = x_train_all / 255.0, x_test / 255.0 

In [0]:
 x_train_all[0][0][0][0]

In [0]:
x_train_all = x_train_all.reshape(x_train_all.shape[0], TOTAL_INPUTS)

In [0]:
x_train_all.shape

In [0]:
x_test = x_test.reshape(len(x_test), TOTAL_INPUTS)
print(f'Shape of x_test is {x_test.shape}')

### Create Validation Dataset
50000 imgs = 40000 for training + 10000 validation 
10000 values from 50000 is for validation

In [0]:
x_val = x_train_all[:VALIDATION_SIZE]
y_val = y_train_all[:VALIDATION_SIZE]
x_val.shape

In [0]:
x_train = x_train_all[VALIDATION_SIZE:]
y_train = y_train_all[VALIDATION_SIZE:]
x_train.shape

### Create a small dataset (for illustration)

In [0]:
x_train_xs = x_train[:SMALL_TRAIN_SIZE]
y_train_xs = y_train[:SMALL_TRAIN_SIZE]

# Define the Neural Network using Keras

In [0]:
# hidden layer 'relu', output layer 'softmax'
model_1 = Sequential([
    Dense(units=128, input_dim=TOTAL_INPUTS, activation='relu', name='m1_hidden1'),
    Dense(units=64, activation='relu', name='m1_hidden2'),
    Dense(16, activation='relu', name='m1_hidden3'),
    Dense(10, activation='softmax', name='m1_output')
])

model_1.compile(optimizer='adam', 
                loss='sparse_categorical_crossentropy', 
                metrics=['accuracy'])

In [0]:
model_2 = Sequential()
model_2.add(Dropout(0.2, seed=42, input_shape=(TOTAL_INPUTS,)))
model_2.add(Dense(128, activation='relu', name='m2_hidden1'))
model_2.add(Dense(64, activation='relu', name='m2_hidden2'))
model_2.add(Dense(16, activation='relu', name='m2_hidden3'))
model_2.add(Dense(10, activation='softmax', name='m2_output'))

model_2.compile(optimizer='adam', 
                loss='sparse_categorical_crossentropy', 
                metrics=['accuracy'])

In [0]:
model_3 = Sequential()
model_3.add(Dropout(0.2, seed=42, input_shape=(TOTAL_INPUTS,)))
model_3.add(Dense(128, activation='relu', name='m2_hidden1'))
model_3.add(Dropout(0.25, seed=42))
model_3.add(Dense(64, activation='relu', name='m2_hidden2'))
model_3.add(Dense(16, activation='relu', name='m2_hidden3'))
model_3.add(Dense(10, activation='softmax', name='m2_output'))

model_3.compile(optimizer='adam', 
                loss='sparse_categorical_crossentropy', 
                metrics=['accuracy'])

In [0]:
type(model_1)

In [0]:
model_1.summary() # number of neurons in first second third and last layer and total parameters

# Tensorboard (visualising learning)

In [0]:
def get_tensorboard(model_name):
  
  folder_name = f'{model_name} at {strftime("%H %M")}'
  dir_paths = os.path.join(LOG_DIR, folder_name)

  try:
    os.makedirs(dir_paths)
  except OSError as err:
    print(err.strerror)
  else:
    print('Successfully created directory')

  return TensorBoard(log_dir=dir_paths)

# Fit the Model

In [0]:
samples_per_batch = 1000

In [0]:
# %%time
# nr_epochs = 150

# model_1.fit(x_train_xs, y_train_xs, batch_size=samples_per_batch, epochs=nr_epochs, 
#             callbacks=[get_tensorboard('Model 1')], verbose=0, validation_data=(x_val, y_val))

In [0]:
# %%time
# nr_epochs = 150

# model_2.fit(x_train_xs, y_train_xs, batch_size=samples_per_batch, epochs=nr_epochs, 
#             callbacks=[get_tensorboard('Model 2')], verbose=0, validation_data=(x_val, y_val))

In [0]:
%%time
nr_epochs = 100

model_1.fit(x_train, y_train, batch_size=samples_per_batch, epochs=nr_epochs, 
            callbacks=[get_tensorboard('Model 1 XL')], verbose=0, validation_data=(x_val, y_val))

Successfully created directory
CPU times: user 6min 43s, sys: 6.42 s, total: 6min 50s
Wall time: 4min 1s


In [0]:
%%time
nr_epochs = 100

model_2.fit(x_train, y_train, batch_size=samples_per_batch, epochs=nr_epochs, 
            callbacks=[get_tensorboard('Model 2 XL')], verbose=0, validation_data=(x_val, y_val))

Successfully created directory


In [0]:
%%time
nr_epochs = 100

model_3.fit(x_train, y_train, batch_size=samples_per_batch, epochs=nr_epochs, 
            callbacks=[get_tensorboard('Model 3 XL')], verbose=0, validation_data=(x_val, y_val))

# Predictions on Indivisual Images

In [0]:
x_val[0].shape

In [0]:
test = np.expand_dims(x_val[0], axis=0)
test.shape

In [0]:
np.set_printoptions(precision=3)

In [0]:
model_2.predict(test)

In [0]:
model_2.predict(x_val).shape # 10 predictions of each 10,000

In [0]:
model_2.predict_classes(test)  # predicted classes (largest number among 10 above)

In [0]:
y_val[0]

Loop to print out the actual value and the predicted value of first 10 images in the valuation dataset.

In [0]:
for number in range(10):
  test_img = np.expand_dims(x_val[number], axis=0)
  predicted_val = model_2.predict_classes(test_img)[0]
  print(f'Actual value: {y_val[number][0]} vs. predicted: {predicted_val}')

# Evaluation

In [0]:
model_2.metrics_names

In [0]:
test_loss, test_accuracy = model_2.evaluate(x_test, y_test)
print(f'Test loss is {test_loss:0.3} and test accuracy is {test_accuracy:0.1%}')

## Confusion Matrix

In [0]:
predictions = model_2.predict_classes(x_test)
conf_matrix = confusion_matrix(y_true=y_test, y_pred=predictions)

In [0]:
conf_matrix.shape

In [0]:
nr_rows = conf_matrix.shape[0]
nr_cols = conf_matrix.shape[1]

In [0]:
conf_matrix.max()

In [0]:
conf_matrix.min()

In [0]:
conf_matrix[0]

In [0]:
plt.figure(figsize=(7,7), dpi=100)
plt.imshow(conf_matrix, cmap=plt.cm.Greens)

plt.title('Confusion Matrix', fontsize=16)
plt.ylabel('Actual labels', fontsize=12)
plt.xlabel('Predicted labels', fontsize=12)

tick_marks = np.arange(NR_CLASSES)
plt.yticks(tick_marks, LABEL_NAMES)
plt.xticks(tick_marks, LABEL_NAMES)

plt.colorbar()

for i, j in itertools.product(range(nr_rows), range(nr_cols)):
  plt.text(j, i, conf_matrix[i, j], horizontalalignment='center',
          color='white' if conf_matrix[i, j] > conf_matrix.max()/2 else 'black')

plt.show()

In [0]:
# True Positive
np.diag(conf_matrix)

In [0]:
recall = np.diag(conf_matrix) / np.sum(conf_matrix, axis = 1)
recall

In [0]:
precision = np.diag(conf_matrix) / np.sum(conf_matrix, axis=0)
precision

In [0]:
avg_recall = np.mean(recall)
print(f'Model 2 reacall score is {avg_recall:.2%}')

Average precision for the model as a whole. Calculating the f-score for the model as a whole

In [0]:
avg_precision = np.mean(precision)
print(f'Model 2 precision score is {avg_precision:.2%}')

f1_score = 2 * (avg_precision * avg_recall) / (avg_precision + avg_recall)
print(f'Model 2 f score is {f1_score:.2%}')